In [1]:
import pandas as pd
import pyreadstat as st
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 2T\[MT08-MT13] Microeconometría II\Clases\Stata\jobtraining.dta"

df, meta = st.read_dta(path)
df.head(1)

,train,age,educ,black,hisp,married,earn96,unem96,earn98,unem98
0,0,37,11,1,0,1,0.0,1,1.617924,0


In [6]:
import numpy as np
import statsmodels.api as sm
from scipy import stats

# ---------- Utilidades ----------
def _bootstrap(fn, data, n_boot=1000, random_state=123):
    rng = np.random.default_rng(random_state)
    idx = np.arange(len(data))
    stats_boot = []
    for _ in range(n_boot):
        samp = data.iloc[rng.choice(idx, size=len(idx), replace=True)]
        stats_boot.append(fn(samp))
    return np.array(stats_boot)

def _ipw_stats(df, p_col="p_hat", y_col="earn98", d_col="train"):
    p = df[p_col].to_numpy()
    y = df[y_col].to_numpy()
    d = df[d_col].to_numpy()

    # Pesos estabilizados y normalizados
    w1 = d / p
    w0 = (1 - d) / (1 - p)
    w1 /= w1.sum()
    w0 /= w0.sum()

    mu1 = np.sum(w1 * y)
    mu0 = np.sum(w0 * y)
    ate = mu1 - mu0
    return pd.Series({"mu1": mu1, "mu0": mu0, "ate": ate})

# ---------- Estimación principal ----------
def teffects_ipw(
    df,
    y="earn98",
    d="train",
    x=["age", "educ", "earn96"],
    noconstant=True,
    clip=(1e-6, 1 - 1e-6),
    n_boot=2000,
    random_state=42
):
    # 1) Tratamiento ~ X (logit)
    X = df[x].copy()
    if not noconstant:
        X = sm.add_constant(X, has_constant="add")

    model = sm.Logit(df[d], X)
    res = model.fit(disp=False)

    # 2) Propensión y clipping
    p_hat = res.predict(X)
    if clip is not None:
        low, high = clip
        p_hat = np.clip(p_hat, low, high)

    tmp = df[[y, d]].copy()
    tmp["p_hat"] = p_hat

    # 3) Estadísticos puntuales
    point = _ipw_stats(tmp, p_col="p_hat", y_col=y, d_col=d)

    # 4) Bootstrap (no-paramétrico) para SE, z, p, CI
    def stat_fn(sample_df):
        # Re-estimar p(X) dentro del bootstrap (como hace Stata con "robust"?)
        Xb = sample_df[x].copy()
        if not noconstant:
            Xb = sm.add_constant(Xb, has_constant="add")
        m_b = sm.Logit(sample_df[d], Xb).fit(disp=False)
        p_b = np.clip(m_b.predict(Xb), clip[0], clip[1]) if clip else m_b.predict(Xb)

        df_b = sample_df[[y, d]].copy()
        df_b["p_hat"] = p_b
        s = _ipw_stats(df_b, p_col="p_hat", y_col=y, d_col=d)
        # Devolvemos mu0 y ate (para emular POmean train=0 y ATE)
        return np.array([s["mu0"], s["ate"]])

    boots = _bootstrap(stat_fn, df, n_boot=n_boot, random_state=random_state)
    mu0_boot = boots[:, 0]
    ate_boot = boots[:, 1]

    # SE y z para ATE
    ate_se = ate_boot.std(ddof=1)
    ate_z = point["ate"] / ate_se if ate_se > 0 else np.nan
    ate_p = 2 * (1 - stats.norm.cdf(abs(ate_z)))
    ate_ci = (point["ate"] - 1.96 * ate_se, point["ate"] + 1.96 * ate_se)

    # SE y z para POmean (train=0)
    mu0_se = mu0_boot.std(ddof=1)
    mu0_z = point["mu0"] / mu0_se if mu0_se > 0 else np.nan
    mu0_p = 2 * (1 - stats.norm.cdf(abs(mu0_z)))
    mu0_ci = (point["mu0"] - 1.96 * mu0_se, point["mu0"] + 1.96 * mu0_se)

    out = {
        "ATE": {
            "coef": float(point["ate"]),
            "se": float(ate_se),
            "z": float(ate_z),
            "p": float(ate_p),
            "ci95": ate_ci
        },
        "POmean_train_0": {
            "coef": float(point["mu0"]),
            "se": float(mu0_se),
            "z": float(mu0_z),
            "p": float(mu0_p),
            "ci95": mu0_ci
        },
        "treatment_model": {
            "params": res.params.to_dict(),
            "converged": bool(res.mle_retvals.get("converged", True)),
            "noconstant": bool(noconstant)
        }
    }
    return out

# ---------- Ejecución: ajusta la ruta a tu archivo ----------
# Ruta del .dta (cámbiala por la tuya)
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 2T\[MT08-MT13] Microeconometría II\Clases\Stata\jobtraining.dta"

df = pd.read_stata(path)

# Filtros/limpieza opcional: asegurar binaria train y sin NA en variables usadas
df = df.dropna(subset=["earn98", "train", "age", "educ", "earn96"]).copy()
df["train"] = (df["train"] > 0).astype(int)

res = teffects_ipw(
    df,
    y="earn98",
    d="train",
    x=["age", "educ", "earn96"],
    noconstant=True,      # <- replica ", noconstant" de Stata
    clip=(1e-6, 1 - 1e-6),
    n_boot=2000,
    random_state=123
)

print("Treatment-effects estimation  (IPW ATE)\n")
print("Estimator      : inverse-probability weights")
print("Outcome model  : weighted mean")
print("Treatment model: logit")
print("-" * 78)
print(f"{'earn98':>12} | {'Coefficient':>12}  {'Std. Err.':>10}  {'z':>8}  {'P>|z|':>7}     {'[95% Conf. Interval]':>23}")
print("-" * 78)
ate = res["ATE"]
print(f"{'ATE':>12} | {ate['coef']:12.6f}  {ate['se']:10.6f}  {ate['z']:8.3f}  {ate['p']:7.3f}     ({ate['ci95'][0]:.6f}, {ate['ci95'][1]:.6f})")
mu0 = res["POmean_train_0"]
print(f"{'POmean 0':>12} | {mu0['coef']:12.6f}  {mu0['se']:10.6f}  {mu0['z']:8.3f}  {mu0['p']:7.3f}     ({mu0['ci95'][0]:.6f}, {mu0['ci95'][1]:.6f})")

Treatment-effects estimation  (IPW ATE)

Estimator      : inverse-probability weights
Outcome model  : weighted mean
Treatment model: logit
------------------------------------------------------------------------------
      earn98 |  Coefficient   Std. Err.         z    P>|z|        [95% Conf. Interval]
------------------------------------------------------------------------------
         ATE |     1.046882    0.494240     2.118    0.034     (0.078171, 2.015593)
    POmean 0 |     9.174996    0.267456    34.305    0.000     (8.650783, 9.699210)


In [19]:
import numpy as np
import statsmodels.api as sm
from scipy import stats

# ---------- IPW con pesos normalizados ----------
def _ipw_group_means(y, d, p):
    # Pesos no estabilizados (ATE) y normalización por grupo
    w1 = d / p
    w0 = (1 - d) / (1 - p)
    w1 = w1 / w1.sum()
    w0 = w0 / w0.sum()
    mu1 = np.sum(w1 * y)
    mu0 = np.sum(w0 * y)
    return mu1, mu0, mu1 - mu0

def _fit_logit(df, d, x, noconstant=True):
    X = df[x].copy()
    if not noconstant:
        X = sm.add_constant(X, has_constant="add")
    res = sm.Logit(df[d].values, X.values).fit(disp=False)
    p = res.predict(X.values)
    return res, p

def teffects_ipw_python(
    df,
    y="earn98",
    d="train",
    x=("age", "educ", "earn96"),
    noconstant=True,
    clip=(1e-6, 1 - 1e-6),
    var_method="bootstrap",   # "bootstrap" o "none"
    n_boot=2000,
    random_state=123
):
    # Ajuste del modelo de tratamiento
    res, p = _fit_logit(df, d, x, noconstant=noconstant)
    if clip is not None:
        p = np.clip(p, clip[0], clip[1])

    yv = df[y].to_numpy(dtype=float)
    dv = df[d].to_numpy(dtype=float)

    mu1, mu0, ate = _ipw_group_means(yv, dv, p)

    # Varianza/SE
    if var_method == "bootstrap":
        rng = np.random.default_rng(random_state)
        n = len(df)
        ate_b = np.empty(n_boot)
        mu0_b = np.empty(n_boot)
        idx = np.arange(n)

        for b in range(n_boot):
            ii = rng.choice(idx, size=n, replace=True)
            df_b = df.iloc[ii]

            res_b, p_b = _fit_logit(df_b, d, x, noconstant=noconstant)
            if clip is not None:
                p_b = np.clip(p_b, clip[0], clip[1])

            yb = df_b[y].to_numpy(dtype=float)
            db = df_b[d].to_numpy(dtype=float)

            mu1_b, mu0_b[b], ate_b[b] = _ipw_group_means(yb, db, p_b)

        ate_se = ate_b.std(ddof=1)
        mu0_se = mu0_b.std(ddof=1)
    else:
        # Sin varianza (placeholder); podés implementar aquí una sándwich IF-based si querés calc analítico.
        ate_se = np.nan
        mu0_se = np.nan

    # Estadísticos
    def zpa(ci_est, se):
        if not np.isfinite(se) or se == 0:
            return np.nan, np.nan, (np.nan, np.nan)
        z = ci_est / se
        pval = 2 * (1 - stats.norm.cdf(abs(z)))
        ci = (ci_est - 1.96 * se, ci_est + 1.96 * se)
        return z, pval, ci

    ate_z, ate_p, ate_ci = zpa(ate, ate_se)
    mu0_z, mu0_p, mu0_ci = zpa(mu0, mu0_se)

    # Salida estilo Stata
    print("Treatment-effects estimation  (IPW ATE)\n")
    print("Estimator      : inverse-probability weights")
    print("Outcome model  : weighted mean")
    print("Treatment model: logit")
    print("-" * 78)
    print(f"{y:>12} |  {'Coefficient':>11}   {'Std. Err.':>9}  {'z':>8}  {'P>|z|':>7}        {'[95% Conf. Interval]':>23}")
    print("-" * 78)
    print(f"{'ATE':>12} |  {ate:11.6f}   {ate_se:9.6f}  {ate_z:8.3f}  {ate_p:7.3f}     ({ate_ci[0]:.6f}, {ate_ci[1]:.6f})")
    print(f"{'POmean 0':>12} |  {mu0:11.6f}   {mu0_se:9.6f}  {mu0_z:8.3f}  {mu0_p:7.3f}     ({mu0_ci[0]:.6f}, {mu0_ci[1]:.6f})")

    return {
        "ATE": dict(coef=ate, se=ate_se, z=ate_z, p=ate_p, ci95=ate_ci),
        "POmean_train_0": dict(coef=mu0, se=mu0_se, z=mu0_z, p=mu0_p, ci95=mu0_ci),
        "treatment_model": dict(params=dict(zip(['const']*(not noconstant)+list(x), res.params)),
                                converged=bool(res.mle_retvals.get("converged", True)),
                                noconstant=bool(noconstant))
    }

# --------- Ejemplo de uso ----------

df = pd.read_stata(path).dropna(subset=["earn98","train","age","educ","earn96"]).copy()
df["train"] = (df["train"] > 0).astype(int)
_ = teffects_ipw_python(
    df,
    y="earn98",
    d="train",
    x=["age", "educ", "earn96"],   # <--- lista, no tupla
    noconstant=True,
    var_method="bootstrap",
    n_boot=5000,
    random_state=2025
)


Treatment-effects estimation  (IPW ATE)

Estimator      : inverse-probability weights
Outcome model  : weighted mean
Treatment model: logit
------------------------------------------------------------------------------
      earn98 |  Coefficient   Std. Err.         z    P>|z|           [95% Conf. Interval]
------------------------------------------------------------------------------
         ATE |     1.046882    0.482386     2.170    0.030     (0.101406, 1.992358)
    POmean 0 |     9.174996    0.274338    33.444    0.000     (8.637293, 9.712700)


In [20]:
import numpy as np
import statsmodels.api as sm
from scipy import stats

def _logit_fit_pscore(df, d, x, noconstant=True):
    X = df[list(x)].to_numpy(dtype=float)
    if not noconstant:
        X = sm.add_constant(X, has_constant="add")
    y = df[d].to_numpy(dtype=float)
    mod = sm.Logit(y, X)
    res = mod.fit(disp=False)
    p = res.predict(X)
    return res, X, y, p

def _ipw_normalized_means(y, d, p, clip=None):
    if clip is not None:
        p = np.clip(p, clip[0], clip[1])

    a = d / p
    b = (1 - d) / (1 - p)
    S1 = a.sum()
    S0 = b.sum()
    w1 = a / S1
    w0 = b / S0

    mu1 = np.sum(w1 * y)
    mu0 = np.sum(w0 * y)
    ate = mu1 - mu0
    return mu1, mu0, ate, (w1, w0, S1, S0)

def _if_beta_per_obs(X, d, p):
    """
    Influence function de beta (logit MLE):
      IF_i(beta) = J^{-1} s_i,  con s_i = x_i (d_i - p_i),
      J = (1/n) X' W X, W = diag(p_i (1-p_i))
    Devuelve matriz n x k con cada fila IF_i(beta).
    """
    n, k = X.shape
    w = p * (1 - p)
    WX = X * w[:, None]
    J = (X.T @ WX) / n
    J_inv = np.linalg.inv(J)
    scores = X * (d - p)[:, None]  # n x k
    IF_beta = scores @ J_inv.T      # n x k
    return IF_beta  # n x k

def _grad_mu_normalized_beta(X, y, d, p, mu, group="treat1"):
    """
    Gradiente d mu_g / d beta para IPW normalizado.
      Para g=1: a_i = d_i/p_i,  ∂a_i/∂β = - d_i * (1-p_i)/p_i * x_i
      Para g=0: b_i = (1-d_i)/(1-p_i), ∂b_i/∂β = (1-d_i) * p_i/(1-p_i) * x_i
      d mu_g / dβ = (1/Sg) sum_i ( ∂w_i/∂β * y_i ), con w_i = a_i/S1 o b_i/S0
                   = (1/Sg) sum_i [ ∂a_i/∂β (y_i - mu_g) ]  (o análogo para b_i)
    Retorna vector k.
    """
    X = np.asarray(X)
    y = np.asarray(y)
    d = np.asarray(d)
    p = np.asarray(p)
    n, k = X.shape

    if group == "treat1":
        # g=1
        # a_i = d/p
        S = np.sum(d / p)
        # ∂a_i/∂β = - d_i*(1-p_i)/p_i * x_i
        coef = - d * (1 - p) / p
        resid = (y - mu)  # n,
        # sum_i ∂a_i/∂β * (y_i - mu) = sum_i [ coef_i * resid_i * x_i ]
        term = (coef * resid)[:, None] * X
        grad = term.sum(axis=0) / S
        return grad  # k,
    else:
        # g=0
        # b_i = (1-d)/(1-p)
        S = np.sum((1 - d) / (1 - p))
        # ∂b_i/∂β = (1-d_i) * p_i/(1-p_i) * x_i
        coef = (1 - d) * p / (1 - p)
        resid = (y - mu)
        term = (coef * resid)[:, None] * X
        grad = term.sum(axis=0) / S
        return grad  # k,

def _sandwich_se_ipw_normalized(X, y, d, p, clip=None):
    """
    Calcula SE analíticos sándwich para:
      - mu1_hat (IPW normalizado sobre tratados)
      - mu0_hat (IPW normalizado sobre no tratados)
      - ate_hat  = mu1_hat - mu0_hat
    """
    n = len(y)
    if clip is not None:
        p = np.clip(p, clip[0], clip[1])

    mu1, mu0, ate, (w1, w0, S1, S0) = _ipw_normalized_means(y, d, p, clip=None)

    # IF directas (teniendo beta fija):
    #   IF_i(mu1) = w1_i * (y_i - mu1)
    #   IF_i(mu0) = w0_i * (y_i - mu0)
    IF_mu1_direct = w1 * (y - mu1)
    IF_mu0_direct = w0 * (y - mu0)
    IF_ate_direct = IF_mu1_direct - IF_mu0_direct

    # Influence de beta:
    IF_beta = _if_beta_per_obs(X, d, p)  # n x k

    # Gradientes d mu_g / d beta
    g1 = _grad_mu_normalized_beta(X, y, d, p, mu1, group="treat1")  # k,
    g0 = _grad_mu_normalized_beta(X, y, d, p, mu0, group="treat0")  # k,
    gA = g1 - g0

    # Termino indirecto via beta: (∂mu/∂β)' IF_i(β)
    IF_mu1 = IF_mu1_direct + IF_beta @ g1
    IF_mu0 = IF_mu0_direct + IF_beta @ g0
    IF_ate = IF_ate_direct + IF_beta @ gA

    # Varianzas asintóticas: Var(theta_hat) ≈ Var(IF)/n
    def _se(IF):
        IFc = IF - IF.mean()   # centrar por seguridad
        var = (IFc @ IFc) / n  # promedio de cuadrados
        return np.sqrt(var / n)

    se_mu1 = _se(IF_mu1)
    se_mu0 = _se(IF_mu0)
    se_ate = _se(IF_ate)
    return (mu1, se_mu1), (mu0, se_mu0), (ate, se_ate)

def teffects_ipw_sandwich(
    df,
    y="earn98",
    d="train",
    x=("age", "educ", "earn96"),
    noconstant=True,
    clip=(1e-6, 1 - 1e-6),
):
    # 1) Logit para p(X)
    res, X, D, p = _logit_fit_pscore(df, d, x, noconstant=noconstant)
    if clip is not None:
        p = np.clip(p, clip[0], clip[1])
    Y = df[y].to_numpy(dtype=float)

    # 2) Puntos y SE analíticos (sándwich) para IPW normalizado
    (mu1, se_mu1), (mu0, se_mu0), (ate, se_ate) = _sandwich_se_ipw_normalized(
        X, Y, D, p, clip=None
    )

    # 3) Estadísticos
    def zpa(theta, se):
        z = theta / se
        pval = 2 * (1 - stats.norm.cdf(abs(z)))
        ci = (theta - 1.96 * se, theta + 1.96 * se)
        return z, pval, ci

    z_ate, p_ate, ci_ate = zpa(ate, se_ate)
    z_mu0, p_mu0, ci_mu0 = zpa(mu0, se_mu0)

    # 4) Salida estilo Stata
    print("Treatment-effects estimation  (IPW ATE)\n")
    print("Estimator      : inverse-probability weights")
    print("Outcome model  : weighted mean")
    print("Treatment model: logit")
    print("-" * 82)
    header = f"{y:>12} |  {'Coefficient':>11}   {'Std. Err.':>9}  {'z':>8}  {'P>|z|':>7}           {'[95% Conf. Interval]':>23}"
    print(header)
    print("-" * 82)
    print(f"{'ATE':>12} |  {ate:11.6f}   {se_ate:9.6f}  {z_ate:8.3f}  {p_ate:7.3f}     ({ci_ate[0]:.6f}, {ci_ate[1]:.6f})")
    print(f"{'POmean 0':>12} |  {mu0:11.6f}   {se_mu0:9.6f}  {z_mu0:8.3f}  {p_mu0:7.3f}     ({ci_mu0[0]:.6f}, {ci_mu0[1]:.6f})")

    # (Opcional) podrías imprimir POmean 1 también:
    # z_mu1, p_mu1, ci_mu1 = zpa(mu1, se_mu1)
    # print(f"{'POmean 1':>12} |  {mu1:11.6f}   {se_mu1:9.6f}  {z_mu1:8.3f}  {p_mu1:7.3f}     ({ci_mu1[0]:.6f}, {ci_mu1[1]:.6f})")

    return {
        "ATE": dict(coef=float(ate), se=float(se_ate), z=float(z_ate), p=float(p_ate), ci95=ci_ate),
        "POmean_train_0": dict(coef=float(mu0), se=float(se_mu0), z=float(z_mu0), p=float(p_mu0), ci95=ci_mu0),
        "POmean_train_1": dict(coef=float(mu1), se=float(se_mu1)),
        "treatment_model": dict(params=res.params.tolist(), converged=bool(res.mle_retvals.get("converged", True)), noconstant=bool(noconstant))
    }

# ---------------- Ejemplo de uso ----------------
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 2T\[MT08-MT13] Microeconometría II\Clases\Stata\jobtraining.dta"
df = pd.read_stata(path).dropna(subset=["earn98","train","age","educ","earn96"]).copy()
df["train"] = (df["train"] > 0).astype(int)
_ = teffects_ipw_sandwich(df, y="earn98", d="train", x=("age","educ","earn96"), noconstant=True, clip=(1e-6, 1-1e-6))

Treatment-effects estimation  (IPW ATE)

Estimator      : inverse-probability weights
Outcome model  : weighted mean
Treatment model: logit
----------------------------------------------------------------------------------
      earn98 |  Coefficient   Std. Err.         z    P>|z|              [95% Conf. Interval]
----------------------------------------------------------------------------------
         ATE |     1.046882    0.267204     3.918    0.000     (0.523163, 1.570602)
    POmean 0 |     9.174996    0.130275    70.428    0.000     (8.919657, 9.430336)


In [21]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
import statsmodels.api as sm
from scipy.stats import norm

@dataclass
class AIPWResult:
    ate: float
    ate_se: float
    ate_ci: tuple
    ate_z: float
    ate_p: float
    pomean0: float
    pomean0_se: float
    pomean0_ci: tuple
    pomean0_z: float
    pomean0_p: float

def _add_intercept(X: np.ndarray):
    return np.column_stack([np.ones(X.shape[0]), X])

def _clip01(p, eps=1e-6):
    return np.clip(p, eps, 1-eps)

def teffects_aipw(df: pd.DataFrame,
                  y_col: str = "earn98",
                  d_col: str = "train",
                  x_cols: list = ("age", "educ", "earn96"),
                  treatment_noconstant: bool = True) -> AIPWResult:
    """
    Replicates: teffects aipw (Y Xs) (D Xs, noconstant)
      - treatment_noconstant=True -> logit without intercept (as in Stata command)
      - outcome models are linear (OLS) with intercepts, fit separately by D.
    """

    # Extract arrays
    Y = df[y_col].astype(float).to_numpy()
    D = df[d_col].astype(int).to_numpy()
    X = df[list(x_cols)].astype(float).to_numpy()
    n = len(Y)

    # --- Propensity score model: logit(D ~ X) ---
    Xt = X if treatment_noconstant else _add_intercept(X)
    logit = sm.Logit(D, Xt).fit(disp=False)
    p = _clip01(logit.predict(Xt))

    # --- Outcome models: OLS with intercepts, fit separately in treated/controls ---
    X1 = _add_intercept(X[D == 1])
    X0 = _add_intercept(X[D == 0])
    y1 = Y[D == 1]
    y0 = Y[D == 0]

    ols1 = sm.OLS(y1, X1).fit()
    ols0 = sm.OLS(y0, X0).fit()

    m1 = ols1.predict(_add_intercept(X))  # m1(X)
    m0 = ols0.predict(_add_intercept(X))  # m0(X)

    # --- AIPW estimators ---
    # ATE EIF for each i:
    psi_ate = (m1 - m0) + D * (Y - m1) / p - (1 - D) * (Y - m0) / (1 - p)
    ate_hat = psi_ate.mean()
    ate_se = psi_ate.std(ddof=1) / np.sqrt(n)

    # POmean for D=0 (mu0) and its EIF:
    # EIF(mu0) = m0(X) + (1-D)*(Y - m0(X))/(1-p) - mu0
    phi_mu0 = m0 + (1 - D) * (Y - m0) / (1 - p)
    mu0_hat = phi_mu0.mean()
    mu0_se = phi_mu0.std(ddof=1) / np.sqrt(n)

    # Inference
    def infer(est, se):
        z = est / se
        pval = 2 * (1 - norm.cdf(abs(z)))
        ci = (est - 1.96 * se, est + 1.96 * se)
        return z, pval, ci

    ate_z, ate_p, ate_ci = infer(ate_hat, ate_se)
    mu0_z, mu0_p, mu0_ci = infer(mu0_hat, mu0_se)

    return AIPWResult(
        ate=ate_hat, ate_se=ate_se, ate_ci=ate_ci, ate_z=ate_z, ate_p=ate_p,
        pomean0=mu0_hat, pomean0_se=mu0_se, pomean0_ci=mu0_ci, pomean0_z=mu0_z, pomean0_p=mu0_p
    )

def print_teffects_table(res: AIPWResult, n_obs: int):
    # Format similar to Stata output (rounded sensibly)
    def fmt(x): return f"{x:>10.6f}"
    def fmti(x): return f"{x:>10.6f}"
    print("Treatment-effects estimation".ljust(46) + f"Number of obs     = {n_obs:>10}")
    print("Estimator      : augmented IPW")
    print("Outcome model  : linear by ML")
    print("Treatment model: logit")
    print("-" * 78)
    print(f"{'':13}|{'':15}Robust")
    print(f"{'earn98':>13} | {'Coefficient':>11}  {'std. err.':>10}  {'z':>6}  {'P>|z|':>6}  {'[95% conf. interval]':>23}")
    print("-" * 78)
    # ATE row (train: 1 vs 0)
    print(f"{'ATE':>13} |")
    row = [
        "   (1 vs 0) ",
        fmt(res.ate), fmt(res.ate_se),
        f"{res.ate_z:6.2f}", f"{res.ate_p:6.3f}",
        f"{fmti(res.ate_ci[0])}    {fmti(res.ate_ci[1])}"
    ]
    print(f"{'train':>11} |{row[0]:>11}{row[1]:>12}{row[2]:>12}{row[3]:>7}{row[4]:>7}     {row[5]}")
    print("-" * 78)
    # POmean 0 row
    print(f"{'POmean':>13} |")
    row0 = [
        "          0 ",
        fmt(res.pomean0), fmt(res.pomean0_se),
        f"{res.pomean0_z:6.2f}", f"{res.pomean0_p:6.3f}",
        f"{fmti(res.pomean0_ci[0])}    {fmti(res.pomean0_ci[1])}"
    ]
    print(f"{'train':>11} |{row0[0]:>11}{row0[1]:>12}{row0[2]:>12}{row0[3]:>7}{row0[4]:>7}     {row0[5]}")
    print("-" * 78)

# ---------------------------
# Example usage (adjust df):
# df must contain: 'earn98' (Y), 'train' (D in {0,1}), covariates: 'age','educ','earn96'
# ---------------------------
df = pd.read_stata(path).dropna(subset=["earn98","train","age","educ","earn96"]).copy()
df["train"] = (df["train"] > 0).astype(int)
res = teffects_aipw(df, y_col="earn98", d_col="train",
                     x_cols=["age", "educ", "earn96"],
                     treatment_noconstant=True)
print_teffects_table(res, n_obs=len(df))


Treatment-effects estimation                  Number of obs     =       1130
Estimator      : augmented IPW
Outcome model  : linear by ML
Treatment model: logit
------------------------------------------------------------------------------
             |               Robust
       earn98 | Coefficient   std. err.       z   P>|z|     [95% conf. interval]
------------------------------------------------------------------------------
          ATE |
      train |   (1 vs 0)     2.880591    0.430435   6.69  0.000       2.036938      3.724244
------------------------------------------------------------------------------
       POmean |
      train |          0     9.228853    0.263368  35.04  0.000       8.712651      9.745055
------------------------------------------------------------------------------


In [23]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
import statsmodels.api as sm
from scipy.stats import norm

# =========================
# Núcleo AIPW (plug-in)
# =========================
@dataclass
class AIPWResult:
    ate: float
    ate_se: float
    ate_ci: tuple
    ate_z: float
    ate_p: float
    pomean0: float
    pomean0_se: float
    pomean0_ci: tuple
    pomean0_z: float
    pomean0_p: float
    B: int = 0
    se_source: str = "influence"  # "bootstrap" cuando usemos Opción 1

def _add_intercept(X: np.ndarray):
    return np.column_stack([np.ones(X.shape[0]), X])

def _clip01(p, eps=1e-6):
    return np.clip(p, eps, 1-eps)

def teffects_aipw(df: pd.DataFrame,
                  y_col: str = "earn98",
                  d_col: str = "train",
                  x_cols: list = ("age", "educ", "earn96"),
                  treatment_noconstant: bool = True) -> AIPWResult:
    """
    AIPW para: teffects aipw (Y Xs) (D Xs, noconstant)
    - logit(D ~ X) sin constante si treatment_noconstant=True
    - OLS por separado con intercepto para Y|D=1 y Y|D=0
    - Devuelve ATE y POmean (train=0) con SE plug-in (influence simple)
    """
    Y = df[y_col].astype(float).to_numpy()
    D = df[d_col].astype(int).to_numpy()
    X = df[list(x_cols)].astype(float).to_numpy()
    n = len(Y)

    # Propensity: logit(D ~ X)
    Xt = X if treatment_noconstant else _add_intercept(X)
    logit = sm.Logit(D, Xt).fit(disp=False)
    p = _clip01(logit.predict(Xt))

    # Outcome models con intercepto
    X1 = _add_intercept(X[D == 1])
    X0 = _add_intercept(X[D == 0])
    y1 = Y[D == 1]
    y0 = Y[D == 0]

    ols1 = sm.OLS(y1, X1).fit()
    ols0 = sm.OLS(y0, X0).fit()

    m1 = ols1.predict(_add_intercept(X))
    m0 = ols0.predict(_add_intercept(X))

    # EIFs y estimaciones
    psi_ate = (m1 - m0) + D * (Y - m1) / p - (1 - D) * (Y - m0) / (1 - p)
    ate_hat = psi_ate.mean()
    ate_se = psi_ate.std(ddof=1) / np.sqrt(n)

    phi_mu0 = m0 + (1 - D) * (Y - m0) / (1 - p)
    mu0_hat = phi_mu0.mean()
    mu0_se = phi_mu0.std(ddof=1) / np.sqrt(n)

    def infer(est, se):
        z = est / se
        pval = 2 * (1 - norm.cdf(abs(z)))
        ci = (est - 1.96 * se, est + 1.96 * se)
        return z, pval, ci

    ate_z, ate_p, ate_ci = infer(ate_hat, ate_se)
    mu0_z, mu0_p, mu0_ci = infer(mu0_hat, mu0_se)

    return AIPWResult(
        ate=ate_hat, ate_se=ate_se, ate_ci=ate_ci, ate_z=ate_z, ate_p=ate_p,
        pomean0=mu0_hat, pomean0_se=mu0_se, pomean0_ci=mu0_ci, pomean0_z=mu0_z, pomean0_p=mu0_p
    )

# =========================
# Opción 1: Bootstrap SEs
# =========================
def _aipw_point_estimates(df, y_col, d_col, x_cols, treatment_noconstant):
    """Devuelve (ATE, POmean0) para una muestra dada, re-ajustando todo."""
    Y = df[y_col].astype(float).to_numpy()
    D = df[d_col].astype(int).to_numpy()
    X = df[list(x_cols)].astype(float).to_numpy()

    Xt = X if treatment_noconstant else np.column_stack([np.ones(X.shape[0]), X])
    p = _clip01(sm.Logit(D, Xt).fit(disp=False).predict(Xt))

    X1 = _add_intercept(X[D == 1]); y1 = Y[D == 1]
    X0 = _add_intercept(X[D == 0]); y0 = Y[D == 0]
    ols1 = sm.OLS(y1, X1).fit(); ols0 = sm.OLS(y0, X0).fit()

    m1 = ols1.predict(_add_intercept(X))
    m0 = ols0.predict(_add_intercept(X))

    ate = np.mean((m1 - m0) + D*(Y - m1)/p - (1 - D)*(Y - m0)/(1 - p))
    mu0 = np.mean(m0 + (1 - D)*(Y - m0)/(1 - p))
    return ate, mu0

def teffects_aipw_bootstrap(df: pd.DataFrame,
                            y_col: str = "earn98",
                            d_col: str = "train",
                            x_cols: list = ("age", "educ", "earn96"),
                            treatment_noconstant: bool = True,
                            B: int = 500,
                            seed: int = 123) -> AIPWResult:
    """
    Calcula SEs por bootstrap para ATE y POmean0.
    - Re-muestrea filas con reemplazo y re-ajusta logit + OLS en cada réplica.
    - Devuelve las mismas métricas (z, p, CI) usando SE_bootstrap (normal-approx).
    """
    # Estimación puntual en muestra completa
    base_res = teffects_aipw(df, y_col, d_col, x_cols, treatment_noconstant)
    ate_hat, mu0_hat = base_res.ate, base_res.pomean0

    # Bootstrap
    rng = np.random.default_rng(seed)
    n = len(df)
    ate_b = np.empty(B); mu0_b = np.empty(B)

    for b in range(B):
        idx = rng.integers(low=0, high=n, size=n)
        samp = df.iloc[idx].reset_index(drop=True)
        ate_b[b], mu0_b[b] = _aipw_point_estimates(
            samp, y_col, d_col, x_cols, treatment_noconstant
        )

    ate_se = ate_b.std(ddof=1)
    mu0_se = mu0_b.std(ddof=1)

    # Inferencia normal (como Stata reporta en teffects)
    def infer(est, se):
        z = est / se
        pval = 2 * (1 - norm.cdf(abs(z)))
        ci = (est - 1.96 * se, est + 1.96 * se)
        return z, pval, ci

    ate_z, ate_p, ate_ci = infer(ate_hat, ate_se)
    mu0_z, mu0_p, mu0_ci = infer(mu0_hat, mu0_se)

    return AIPWResult(
        ate=ate_hat, ate_se=ate_se, ate_ci=ate_ci, ate_z=ate_z, ate_p=ate_p,
        pomean0=mu0_hat, pomean0_se=mu0_se, pomean0_ci=mu0_ci, pomean0_z=mu0_z, pomean0_p=mu0_p,
        B=B, se_source="bootstrap"
    )

# =========================
# Impresión estilo Stata
# =========================
def print_teffects_table(res: AIPWResult, n_obs: int):
    def fmt(x): return f"{x:>10.6f}"
    def fmti(x): return f"{x:>10.6f}"
    print(f"Treatment-effects estimation".ljust(46) + f"Number of obs     = {n_obs:>10}")
    print("Estimator      : augmented IPW")
    print("Outcome model  : linear by ML")
    print("Treatment model: logit")
    label = "Robust (bootstrap B={})".format(res.B) if res.se_source == "bootstrap" else "Robust"
    print("-" * 78)
    print(f"{'':13}|{label:>27}")
    print(f"{'earn98':>13} | {'Coefficient':>11}  {'std. err.':>10}  {'z':>6}  {'P>|z|':>6}  {'[95% conf. interval]':>23}")
    print("-" * 78)
    # ATE
    print(f"{'ATE':>13} |")
    row = [
        "   (1 vs 0) ",
        fmt(res.ate), fmt(res.ate_se),
        f"{res.ate_z:6.2f}", f"{res.ate_p:6.3f}",
        f"{fmti(res.ate_ci[0])}    {fmti(res.ate_ci[1])}"
    ]
    print(f"{'train':>11} |{row[0]:>11}{row[1]:>12}{row[2]:>12}{row[3]:>7}{row[4]:>7}     {row[5]}")
    print("-" * 78)
    # POmean 0
    print(f"{'POmean':>13} |")
    row0 = [
        "          0 ",
        fmt(res.pomean0), fmt(res.pomean0_se),
        f"{res.pomean0_z:6.2f}", f"{res.pomean0_p:6.3f}",
        f"{fmti(res.pomean0_ci[0])}    {fmti(res.pomean0_ci[1])}"
    ]
    print(f"{'train':>11} |{row0[0]:>11}{row0[1]:>12}{row0[2]:>12}{row0[3]:>7}{row0[4]:>7}     {row0[5]}")
    print("-" * 78)

# =========================
# Ejemplo de uso
# =========================
# df: DataFrame con columnas 'earn98' (Y), 'train' (D en {0,1}), 'age','educ','earn96'.
df = pd.read_stata(path).dropna(subset=["earn98","train","age","educ","earn96"]).copy()
df["train"] = (df["train"] > 0).astype(int)
res = teffects_aipw_bootstrap(
    df,
    y_col="earn98",
    d_col="train",
    x_cols=["age", "educ", "earn96"],
    treatment_noconstant=True,
    B=1000,   # aumentar para mayor precisión
    seed=123
)
print_teffects_table(res, n_obs=len(df))


Treatment-effects estimation                  Number of obs     =       1130
Estimator      : augmented IPW
Outcome model  : linear by ML
Treatment model: logit
------------------------------------------------------------------------------
             |  Robust (bootstrap B=1000)
       earn98 | Coefficient   std. err.       z   P>|z|     [95% conf. interval]
------------------------------------------------------------------------------
          ATE |
      train |   (1 vs 0)     2.880591    0.617592   4.66  0.000       1.670112      4.091071
------------------------------------------------------------------------------
       POmean |
      train |          0     9.228853    0.271437  34.00  0.000       8.696836      9.760870
------------------------------------------------------------------------------


In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression, LinearRegression
from scipy.stats import norm  # ✅ import correcto para cdf
from sklearn.utils import resample

# -------------------------------------------------------------
# 1. Datos
# -------------------------------------------------------------
# Ejemplo: df = pd.read_stata('lalonde.dta')
# Variables: earn98 (Y), train (D), age educ earn96 (X)

df = pd.read_stata(path).dropna(subset=["earn98","train","age","educ","earn96"]).copy()
df["train"] = (df["train"] > 0).astype(int)

y = df['earn98'].to_numpy()
t = df['train'].to_numpy()
X = df[['age', 'educ', 'earn96']].to_numpy()

# -------------------------------------------------------------
# 2. Modelo de tratamiento (propensity score)
# -------------------------------------------------------------
logit = LogisticRegression(fit_intercept=False, solver='lbfgs')
ps = logit.fit(X, t).predict_proba(X)[:, 1]
ps = np.clip(ps, 1e-6, 1 - 1e-6)  # evitar extremos

# -------------------------------------------------------------
# 3. Modelo de outcome por grupo
# -------------------------------------------------------------
reg_treat = LinearRegression(fit_intercept=True).fit(X[t == 1], y[t == 1])
reg_ctrl  = LinearRegression(fit_intercept=True).fit(X[t == 0], y[t == 0])

mu1 = reg_treat.predict(X)
mu0 = reg_ctrl.predict(X)

# -------------------------------------------------------------
# 4. IPWRA (Augmented IPW)
# -------------------------------------------------------------
ate_i = (t * (y - mu1) / ps) - ((1 - t) * (y - mu0) / (1 - ps)) + (mu1 - mu0)
ate = np.mean(ate_i)

pomean0_i = (1 - t) * (y - mu0) / (1 - ps) + mu0
pomean0 = np.mean(pomean0_i)

# -------------------------------------------------------------
# 5. Varianza robusta tipo sándwich
# -------------------------------------------------------------
v_ate = np.var(ate_i, ddof=1) / len(ate_i)
se_ate = np.sqrt(v_ate)

z_ate = ate / se_ate
p_ate = 2 * (1 - norm.cdf(abs(z_ate)))  # ✅ ahora sí

# -------------------------------------------------------------
# 6. Mostrar resultados
# -------------------------------------------------------------
print("Treatment-effects estimation (IPWRA ATE)")
print("Estimator      : IPW regression adjustment")
print("Outcome model  : linear")
print("Treatment model: logit")
print("-" * 78)
print(f"{'ATE':<10} | {ate:10.6f}   {se_ate:10.6f}   {z_ate:6.2f}   {p_ate:6.3f}")
print(f"{'POmean 0':<10} | {pomean0:10.6f}")


Treatment-effects estimation (IPWRA ATE)
Estimator      : IPW regression adjustment
Outcome model  : linear
Treatment model: logit
------------------------------------------------------------------------------
ATE        |   2.880601     0.430412     6.69    0.000
POmean 0   |   9.228870


In [9]:
import numpy as np
import pandas as pd
from scipy.optimize import root
from scipy.stats import norm

# ------------------------------------------------------------
# Utilidades
# ------------------------------------------------------------
def sigmoid(u):
    # estabilidad numérica
    out = np.empty_like(u)
    pos = u >= 0
    neg = ~pos
    out[pos] = 1.0 / (1.0 + np.exp(-u[pos]))
    e = np.exp(u[neg])
    out[neg] = e / (1.0 + e)
    return out

def clip01(p, eps=1e-8):
    return np.minimum(1 - eps, np.maximum(eps, p))

# ------------------------------------------------------------
# Momentos apilados (EE) tipo GMM/M-estimator
# θ = [α (k), γ1 (k+1), γ0 (k+1), δ (ATE), m0 (POmean0)]
#   α:     coef. logit (sin intercepto)    dimensión k
#   γ1:    OLS tratados (con intercepto)   dimensión k+1
#   γ0:    OLS controles (con intercepto)  dimensión k+1
#   δ:     ATE
#   m0:    POmean(0)
# ------------------------------------------------------------
class IPWRA_MEstimator:
    def __init__(self, df, y_col="earn98", d_col="train", x_cols=("age","educ","earn96"),
                 eps_ps=1e-8):
        self.df = df.dropna(subset=[y_col, d_col] + list(x_cols)).copy()
        self.y = self.df[y_col].to_numpy(float)
        self.d = (self.df[d_col].astype(int) > 0).astype(int).to_numpy()
        self.X = self.df[list(x_cols)].to_numpy(float)             # (n,k)
        self.n, self.k = self.X.shape
        self.Z = np.column_stack([np.ones(self.n), self.X])       # (n,k+1) para OLS con intercepto
        self.eps = eps_ps

    def unpack_theta(self, theta):
        k = self.k
        a  = theta[0:k]                       # α
        g1 = theta[k: k + (k+1)]              # γ1
        g0 = theta[k + (k+1): k + 2*(k+1)]    # γ0
        delta = theta[k + 2*(k+1)]            # δ
        m0    = theta[k + 2*(k+1) + 1]        # m0
        return a, g1, g0, delta, m0

    def moments_i(self, theta):
        """
        Devuelve g_i(θ) de dimensión:
           k                        (logit, sin intercepto)
         + (k+1)                    (OLS tratados)
         + (k+1)                    (OLS controles)
         + 1                        (ATE EIF - δ)
         + 1                        (POmean0 EIF - m0)
        total: 2k + 3 + 2 = 2k + 5
        """
        a, g1, g0, delta, m0 = self.unpack_theta(theta)

        X, Z, y, d, eps = self.X, self.Z, self.y, self.d, self.eps
        k = self.k

        # ps(x; α)  (logit sin constante)
        ps = clip01(sigmoid(X @ a), eps)

        # μ1(x; γ1) y μ0(x; γ0)  (lineales con intercepto)
        mu1 = Z @ g1
        mu0 = Z @ g0

        # Residuos outcome
        e1 = y - mu1
        e0 = y - mu0

        # ----- Bloques de momentos por observación i -----

        # 1) Logit: ∑ x_i (d_i - ps_i) = 0  (k)
        g_logit = X * (d - ps)[:, None]                  # (n,k)

        # 2) OLS tratados: ∑ d_i * z_i * e1_i = 0  (k+1)
        g_ols1 = Z * (d * e1)[:, None]                   # (n,k+1)

        # 3) OLS controles: ∑ (1-d_i) * z_i * e0_i = 0  (k+1)
        g_ols0 = Z * ((1 - d) * e0)[:, None]             # (n,k+1)

        # 4) ATE (EIF): ∑ [ d*(y-μ1)/ps - (1-d)*(y-μ0)/(1-ps) + (μ1-μ0) - δ ] = 0  (1)
        g_ate = (d * (y - mu1) / ps) - ((1 - d) * (y - mu0) / (1 - ps)) + (mu1 - mu0) - delta  # (n,)

        # 5) POmean(0) (EIF): ∑ [ (1-d)*(y-μ0)/(1-ps) + μ0 - m0 ] = 0  (1)
        g_p0  = ((1 - d) * (y - mu0) / (1 - ps)) + mu0 - m0                                   # (n,)

        # Apilar por fila (i) → vector g_i(θ)
        # Orden: [logit(k), ols1(k+1), ols0(k+1), ate(1), p0(1)]
        gi = np.column_stack([
            g_logit,
            g_ols1,
            g_ols0,
            g_ate.reshape(-1,1),
            g_p0.reshape(-1,1)
        ])  # (n, 2k+5)

        return gi

    def moments_mean(self, theta):
        gi = self.moments_i(theta)
        return gi.mean(axis=0)

    # Jacobiano por diferencia finita: A = ∂ \bar g / ∂θ
    def jacobian(self, theta, h=1e-6):
        g0 = self.moments_mean(theta)
        p = theta.size
        A = np.zeros((g0.size, p))
        for j in range(p):
            th = theta.copy()
            step = h * (1.0 + abs(theta[j]))
            th[j] += step
            gj = self.moments_mean(th)
            A[:, j] = (gj - g0) / step
        return A

    def fit(self, init=None, tol=1e-10, verbose=False):
        n, k = self.n, self.k

        # ----- Inicialización razonable -----
        if init is None:
            # Logit (separado) para α (sin intercepto)
            # Usamos mínimos cuadrados para iniciar α (logit con link aproximado)
            # y luego una actualización de Newton simple.
            # Mejor: pseudo-sols con una pasada de IRLS.
            a0 = np.zeros(k)
            # OLS por grupo para γ1 y γ0
            Z = self.Z
            y = self.y
            d = self.d

            # Tratados
            if d.sum() == 0 or d.sum() == n:
                raise ValueError("Tratamiento es degenerado (todo 0 o todo 1).")
            g1_0 = np.linalg.lstsq(Z[d==1], y[d==1], rcond=None)[0]
            g0_0 = np.linalg.lstsq(Z[d==0], y[d==0], rcond=None)[0]

            # Plug-in para δ y m0
            ps0 = clip01(sigmoid(self.X @ a0), self.eps)
            mu1_0 = Z @ g1_0
            mu0_0 = Z @ g0_0

            ate_i0 = d*(y-mu1_0)/ps0 - (1-d)*(y-mu0_0)/(1-ps0) + (mu1_0 - mu0_0)
            delta0 = ate_i0.mean()

            p0_i0 = (1-d)*(y-mu0_0)/(1-ps0) + mu0_0
            m0_0  = p0_i0.mean()

            init = np.concatenate([a0, g1_0, g0_0, [delta0, m0_0]])

        # ----- Resolver EE: mean(g_i(θ)) = 0 -----
        sol = root(lambda th: self.moments_mean(th), init, tol=tol, method='hybr')
        if not sol.success and verbose:
            print("root() no convergió:", sol.message)

        theta_hat = sol.x

        # ----- Varianza sándwich -----
        gi = self.moments_i(theta_hat)                   # (n, q)
        gbar = gi.mean(axis=0)                           # ≈ 0
        B = (gi - gbar).T @ (gi - gbar) / n              # “meat” (q x q)
        A = self.jacobian(theta_hat)                     # (q x p); aquí q==p == 2k+5
        A_inv = np.linalg.inv(A)
        V = (A_inv @ B @ A_inv.T) / n                    # Var(θ_hat)
        se = np.sqrt(np.diag(V))

        # indices de δ y m0
        idx_delta = k + 2*(k+1)
        idx_m0    = idx_delta + 1

        res = {
            "theta": theta_hat,
            "V": V,
            "se": se,
            "A": A,
            "B": B,
            "n": n,
            "k": k,
            "idx": {"alpha": (0, k),
                    "gamma1": (k, k+(k+1)),
                    "gamma0": (k+(k+1), k+2*(k+1)),
                    "delta": idx_delta,
                    "m0": idx_m0}
        }
        return res

def summarize_ipwra_mest(res, est):
    n, k = res["n"], res["k"]
    idx = res["idx"]
    th, se = res["theta"], res["se"]

    delta, se_delta = th[idx["delta"]], se[idx["delta"]]
    z_delta = delta / se_delta
    p_delta = 2*(1 - norm.cdf(abs(z_delta)))

    m0, se_m0 = th[idx["m0"]], se[idx["m0"]]
    z_m0 = m0 / se_m0
    p_m0 = 2*(1 - norm.cdf(abs(z_m0)))

    print("Treatment-effects estimation (IPWRA ATE — stacked M-estimator)")
    print("Estimator      : IPW regression adjustment (joint EE)")
    print("Outcome model  : linear (OLS with intercept)")
    print("Treatment model: logit (no constant)")
    print(f"Number of obs  : {n}")
    print("-"*78)
    print(f"{'ATE':<11}| {delta:11.6f}   {se_delta:10.6f}   {z_delta:6.2f}   {p_delta:6.3f}")
    print(f"{'POmean 0':<11}| {m0:11.6f}   {se_m0:10.6f}   {z_m0:6.2f}   {p_m0:6.3f}")

# ------------------------------------------------------------
# Uso
# ------------------------------------------------------------
# df = pd.read_stata("lalonde.dta")  # o tu DataFrame ya preparado

df = pd.read_stata(path).dropna(subset=["earn98","train","age","educ","earn96"]).copy()
df["train"] = (df["train"] > 0).astype(int)

mest = IPWRA_MEstimator(df, y_col="earn98", d_col="train", x_cols=("age","educ","earn96"))
res  = mest.fit(tol=1e-10, verbose=True)
summarize_ipwra_mest(res, mest)


Treatment-effects estimation (IPWRA ATE — stacked M-estimator)
Estimator      : IPW regression adjustment (joint EE)
Outcome model  : linear (OLS with intercept)
Treatment model: logit (no constant)
Number of obs  : 1130
------------------------------------------------------------------------------
ATE        |    2.880591     0.594358     4.85    0.000
POmean 0   |    9.228853     0.268473    34.38    0.000


In [4]:
# ============================================================
# Endogenous Treatment Effects (Stata: `eteffects`) via stacked GMM
# ============================================================
# Reproduce: eteffects (earn98 age educ earn96) (train age educ earn96)
# Reqs: pip install numpy pandas statsmodels scipy
# Python 3.10+ recomendado

import numpy as np
import pandas as pd
from dataclasses import dataclass
from scipy.stats import norm
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import GMM
from math import erf, sqrt

# -------------------------
# Utilidades
# -------------------------
def mills_terms(eta):
    """Inverse Mills for probit control function."""
    Phi = norm.cdf(eta)
    phi = norm.pdf(eta)
    eps = 1e-12
    l1 = phi / np.clip(Phi, eps, 1 - eps)        # μ(1,Z;γ)
    l0 = -phi / np.clip(1 - Phi, eps, 1 - eps)   # μ(0,Z;γ)
    return l1, l0

def z2p(z):
    return 2 * (1 - 0.5 * (1 + erf(abs(z)/sqrt(2))))

@dataclass
class ETE_GMM_Results:
    n: int
    params: pd.Series            # [alpha, b_age, b_educ, b_earn96, theta_cf, gamma_0..k]
    vcov_gmm: pd.DataFrame
    ate: float
    ate_se_boot: float
    ate_ci: tuple
    pomean0: float
    pomean0_se_boot: float
    pomean0_ci: tuple
    conv: dict

# -------------------------
# Clase GMM con momentos apilados
# -------------------------
class ETE_GMM(GMM):
    """
    Parámetros θ = [α, β (p), θ_cf, γ (k)]
    Momentos:
      m_probit: Z * (D - Φ(Zγ))                 -> k momentos
      m_out   : W * (Y - αD - Xβ - θ_cf μ)      -> (p+2) momentos, W=[1,D,X]
    """
    def __init__(self, endog, exog_out, treat, exog_treat, **kw):
        # Datos
        self.Y = np.asarray(endog).reshape(-1, 1)
        self.D = np.asarray(treat).reshape(-1, 1)
        self.X = np.asarray(exog_out)           # p cols (SIN constante)
        self.Z = np.asarray(exog_treat)         # k cols (CON constante)
        self.W = np.column_stack([np.ones((self.Y.shape[0], 1)), self.D, self.X])
        self.p = self.X.shape[1]
        self.k = self.Z.shape[1]

        # FIX: placeholders para que GMM tenga xnames y no falle
        n = self.Y.shape[0]
        exog_placeholder = np.ones((n, 1))
        instrument_placeholder = np.ones((n, 1))

        start = np.zeros(1 + self.p + 1 + self.k)
        super().__init__(
            endog=np.zeros((n, 1)),
            exog=exog_placeholder,
            instrument=instrument_placeholder,
            k_moms=self.k + (self.p + 2),
            k_params=start.size,
            **kw
        )

    def momcond(self, params):
        alpha = params[0]
        beta  = params[1:1 + self.p]
        theta = params[1 + self.p]
        gamma = params[1 + self.p + 1:]

        eta = self.Z @ gamma
        l1, l0 = mills_terms(eta)
        mu = np.where(self.D.ravel() == 1, l1, l0).reshape(-1, 1)

        # Probit moments
        Phi = norm.cdf(eta).reshape(-1, 1)
        m1 = (self.Z * (self.D - Phi))          # n x k

        # Outcome moments
        yhat = alpha * self.D + self.X @ beta.reshape(-1, 1) + theta * mu
        e = self.Y - yhat                        # n x 1
        m2 = self.W * e                          # n x (p+2)

        return np.column_stack([m1, m2])

# -------------------------
# Estimación principal
# -------------------------
def eteffects_gmm(df, y='earn98', d='train',
                  x_vars=['age','educ','earn96'],
                  z_vars=['age','educ','earn96'],
                  n_boot=1000, random_state=123, maxiter=2, wargs=None):
    # Matrices: X sin constante; Z CON constante (probit)
    Y = df[y].to_numpy()
    D = df[d].astype(int).to_numpy()
    X = df[x_vars].to_numpy()
    Z = sm.add_constant(df[z_vars], has_constant='add').to_numpy()

    # Modelo GMM
    model = ETE_GMM(endog=Y, exog_out=X, treat=D, exog_treat=Z)

    # Inicialización: probit + OLS con CF
    probit_start = sm.Probit(D, Z).fit(disp=False)
    gamma0 = probit_start.params
    eta0 = Z @ gamma0
    l1, l0 = mills_terms(eta0)
    mu0 = np.where(D == 1, l1, l0)

    W_cf = np.column_stack([np.ones_like(D), D, X, mu0])
    ols0 = sm.OLS(Y, W_cf).fit()
    alpha0 = ols0.params[1]
    beta0  = ols0.params[2:2 + X.shape[1]]
    theta0 = ols0.params[-1]
    start = np.r_[alpha0, beta0, theta0, gamma0]

    # Two-step efficient GMM
    res = model.fit(
        start,
        maxiter=maxiter,
        inv_weights=None,
        weights_method='cov',
        optim_method='bfgs',
        optim_args={'disp': False}
    )

    params = res.params
    vcov = res.cov_params()

    # Funcionales (evaluados en θ_hat)
    alpha = params[0]
    beta  = params[1:1 + X.shape[1]]
    theta = params[1 + X.shape[1]]
    gamma = params[1 + X.shape[1] + 1:]

    eta = Z @ gamma
    l1, l0 = mills_terms(eta)
    delta_mu = l1 - l0

    y0_i = (X @ beta) + theta * l0
    ATE = float(alpha + theta * np.mean(delta_mu))
    P0  = float(np.mean(y0_i))

    # Bootstrap para SE de ATE y POmean(0)
    rng = np.random.default_rng(random_state)
    n = len(df)
    ate_bs = np.empty(n_boot)
    p0_bs  = np.empty(n_boot)

    for b in range(n_boot):
        idx = rng.integers(0, n, n)
        dfr = df.iloc[idx].reset_index(drop=True)

        Yb = dfr[y].to_numpy()
        Db = dfr[d].astype(int).to_numpy()
        Xb = dfr[x_vars].to_numpy()
        Zb = sm.add_constant(dfr[z_vars], has_constant='add').to_numpy()

        model_b = ETE_GMM(Yb, Xb, Db, Zb)

        # inicios bootstrap
        probit_b = sm.Probit(Db, Zb).fit(disp=False)
        g0 = probit_b.params
        et0 = Zb @ g0
        l1b, l0b = mills_terms(et0)
        mu0b = np.where(Db == 1, l1b, l0b)
        Wcfb = np.column_stack([np.ones_like(Db), Db, Xb, mu0b])
        ols_b = sm.OLS(Yb, Wcfb).fit()
        s_b = np.r_[ols_b.params[1], ols_b.params[2:2 + Xb.shape[1]], ols_b.params[-1], g0]

        try:
            rb = model_b.fit(
                s_b,
                maxiter=maxiter,
                inv_weights=None,
                weights_method='cov',
                optim_method='bfgs',
                optim_args={'disp': False}
            )
            parb = rb.params
        except Exception:
            parb = s_b  # fallback si no converge

        ab = parb[0]
        bb = parb[1:1 + Xb.shape[1]]
        tb = parb[1 + Xb.shape[1]]
        gb = parb[1 + Xb.shape[1] + 1:]

        etb = Zb @ gb
        l1b, l0b = mills_terms(etb)
        ate_bs[b] = float(ab + tb * np.mean(l1b - l0b))
        p0_bs[b]  = float(np.mean(Xb @ bb + tb * l0b))

    ate_se = float(np.std(ate_bs, ddof=1))
    p0_se  = float(np.std(p0_bs, ddof=1))
    z_95 = 1.959963984540054
    ate_ci = (ATE - z_95*ate_se, ATE + z_95*ate_se)
    p0_ci  = (P0  - z_95*p0_se , P0  + z_95*p0_se )

    return ETE_GMM_Results(
        n=n,
        params=pd.Series(
            params,
            index=(['alpha'] + [f'b_{v}' for v in x_vars] + ['theta_cf'] + [f'gamma_{i}' for i in range(Z.shape[1])])
        ),
        vcov_gmm=pd.DataFrame(vcov),
        ate=ATE, ate_se_boot=ate_se, ate_ci=ate_ci,
        pomean0=P0, pomean0_se_boot=p0_se, pomean0_ci=p0_ci,
        conv={'success': res.converged, 'message': str(res.message)}
    )

# -------------------------
# Pretty print (opcional)
# -------------------------
def print_eteffects(res: ETE_GMM_Results):
    print(f"Endogenous treatment-effects estimation    Number of obs = {res.n}\n")
    print("Outcome model: linear")
    print("Treatment model: probit\n")
    print("--------------------------------------------------------------")
    print("             |               Robust (bootstrap for functionals)")
    print("   earn98    |  Coefficient   Std. Err.       z     P>|z|           [95% Conf. Interval]")
    print("-------------+------------------------------------------------")
    z = res.ate / res.ate_se_boot
    p = z2p(z)
    print(f"ATE (1 vs 0) | {res.ate:10.5f}   {res.ate_se_boot:9.5f}   {z:7.2f}   {p:0.3f}    ({res.ate_ci[0]:.5f}, {res.ate_ci[1]:.5f})")
    print("-------------+------------------------------------------------")
    z0 = res.pomean0 / res.pomean0_se_boot
    p0 = z2p(z0)
    print(f"POmean 0     | {res.pomean0:10.5f}   {res.pomean0_se_boot:9.5f}   {z0:7.2f}   {p0:0.3f}    ({res.pomean0_ci[0]:.5f}, {res.pomean0_ci[1]:.5f})")
    print("--------------------------------------------------------------")
    print("Convergence:", res.conv)

In [5]:
# Cargar datos y limpiar como hiciste

df = pd.read_stata(path).dropna(subset=["earn98","train","age","educ","earn96"]).copy()
df["train"] = (df["train"] > 0).astype(int)

res = eteffects_gmm(df)
print(f"Endogenous treatment-effects estimation    Number of obs = {res.n}\n")
print("Outcome model: linear\nTreatment model: probit\n")
print("--------------------------------------------------------------")
print("             |               Robust (bootstrap for functionals)")
print("   earn98    |  Coefficient   Std. Err.      z     P>|z|")
print("-------------+------------------------------------------------")
z = res.ate / res.ate_se_boot
from math import erf, sqrt
p = 2 * (1 - 0.5 * (1 + erf(abs(z)/sqrt(2))))
print(f"ATE (1 vs 0) | {res.ate:10.5f}   {res.ate_se_boot:9.5f}  {z:6.2f}   {p:0.3f}   [{res.ate_ci[0]:.5f}, {res.ate_ci[1]:.5f}]")
print("-------------+------------------------------------------------")
z0 = res.pomean0 / res.pomean0_se_boot
p0 = 2 * (1 - 0.5 * (1 + erf(abs(z0)/sqrt(2))))
print(f"POmean 0     | {res.pomean0:10.5f}   {res.pomean0_se_boot:9.5f}  {z0:6.2f}   {p0:0.3f}   [{res.pomean0_ci[0]:.5f}, {res.pomean0_ci[1]:.5f}]")
print("--------------------------------------------------------------")
print("Convergence:", res.conv)


LinAlgError: Singular matrix